***GENERATED CODE FOR superstoressarimapred PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***SELECTED MODEL PREDICTION FUNCTION***

In [ ]:
import sklearn.metrics
import mlflow.sklearn
import pyspark
import mlflow
import sklearn
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


class ModelPredictionClass:

    def __init__(self, sparkDf, runID, label, mType, model_path, subtype):
        self.sparkDf = sparkDf
        self.runID = runID
        self.y = label
        self.yTest = []
        self.yPredicted = []
        self.mType = mType
        self.model_path = model_path
        self.subtype = subtype

    def getmetricsRegression(self):
        scoreMetrics = {}
        scoreMetrics["R2"] = round(
            sklearn.metrics.r2_score(self.yTest, self.yPredicted), 1)
        scoreMetrics["Mean_Squared_Error"] = round(
            sklearn.metrics.mean_squared_error(self.yTest, self.yPredicted), 1)

        return scoreMetrics

    def getmetricsClassification(self):
        scoreMetrics = {}
        self.yPredicted = [int(x) for x in self.yPredicted]
        self.yTest = [int(x) for x in self.yTest]
        scoreMetrics["Accuracy"] = round(
            (100 * sklearn.metrics.accuracy_score(y_true=self.yTest, y_pred=self.yPredicted)), 1)
        scoreMetrics["F1"] = round(
            (100 * sklearn.metrics.f1_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        scoreMetrics["Precision"] = round((
            100 * sklearn.metrics.precision_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")),
            1)
        scoreMetrics["Recall"] = round((
            100 * sklearn.metrics.recall_score(y_true=self.yTest, y_pred=self.yPredicted, average="weighted")), 1)
        return scoreMetrics

    def getPrediction(self):
        scoreMetrics = {}
        probabilities = {}
        self.sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        self.sparkDf.toPandas()
        data = self.sparkDf.toPandas()
        complete_model_path = "runs:/" + self.runID + "/" + self.model_path
        self.yTest = list(data[self.y])
        data = data.drop(self.y, axis=1).values
        # model get
        sklearn_model = mlflow.sklearn.load_model(
            model_uri=complete_model_path)
        self.yPredicted = sklearn_model.predict(data).tolist()
        if "classification" == self.mType:
            self.yPredicted = [int(i) for i in self.yPredicted]
            scoreMetrics['result'] = self.getmetricsClassification()

            if self.subtype == "naivebayes" or self.subtype == "multinomialNB" or self.subtype == "GaussianNB" or self.subtype == "ComplementNB" or self.subtype == "BernoulliNB" or self.subtype == "CategoricalNB":
                classes = sklearn_model.classes_.tolist()
                scoreMetrics['classes'] = classes
                for i in range(len(classes)):
                    yproba = sklearn_model.predict_proba(data)[:, i].tolist()
                    my_list_rounded = list(
                        map(lambda x: round(x, ndigits=4), yproba))
                    col_name = "Probability_" + str(classes[i])
                    scoreMetrics[col_name] = my_list_rounded
                    probabilities[col_name] = my_list_rounded
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
            else:
                scoreMetrics['yTest'] = self.yTest
                scoreMetrics['yPredicted'] = self.yPredicted
                return scoreMetrics
        elif "regression" == self.mType:
            scoreMetrics['result'] = self.getmetricsRegression()
            scoreMetrics['yTest'] = self.yTest
            scoreMetrics['yPredicted'] = self.yPredicted
            return scoreMetrics

        elif "clustering" == self.mType:
            self.yPredicted = sklearn_model.predict(data).tolist()
            data['prediction'] = self.yPredicted
            return data


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run superstoressarimapredHooks.ipynb
try:
	#sourcePreExecutionHook()

	sssalecurrent = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1701368057754_SSSalesCurrent.csv', 'filename': 'SSSalesCurrent.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Retail And Marketing/SuperStoresSales/SSSalesCurrent.csv', 'viewFileName': 'SSSalesCurrent.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICTING GIVEN DATA WITH SELECTED MODEL***

In [ ]:
#%run superstoressarimapredHooks.ipynb
try:
	#prediction-pipelinePreExecutionHook()

	sssalecurrent = TransformationMain.run(sssalecurrent,json.dumps( {"FE": [{"transformationsData": [{}], "feature": "Order_Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order_Date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1472.98", "stddev": "2147.85", "min": "3.928", "max": "28106.716", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_First Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "1.55", "min": "-0.189470089", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_First Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Same Day", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.26", "stddev": "0.81", "min": "-0.007506949", "max": "8.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Same Day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Second Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.19", "stddev": "1.87", "min": "-0.661398746", "max": "12.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Second Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Standard Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.86", "stddev": "3.39", "min": "-0.483581511", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Standard Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Consumer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.28", "stddev": "3.25", "min": "-0.534658316", "max": "19.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Consumer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Corporate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.92", "stddev": "2.24", "min": "-0.209610384", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Corporate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Home Office", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.01", "stddev": "1.66", "min": "-0.308929391", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Home Office"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Country_United States", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.2", "stddev": "4.68", "min": "0.600559077", "max": "27.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Country_United States"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_Central", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.44", "stddev": "2.13", "min": "-0.475643886", "max": "17.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_Central"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_East", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.67", "stddev": "2.26", "min": "-0.623750938", "max": "13.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_East"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_South", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.06", "stddev": "1.62", "min": "-0.094892784", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_South"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_West", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.04", "stddev": "2.47", "min": "-0.395274925", "max": "12.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_West"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Furniture", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.36", "stddev": "1.43", "min": "-0.018243082", "max": "8.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Furniture"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Office Supplies", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.71", "stddev": "2.98", "min": "-0.339093471", "max": "17.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Office Supplies"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Technology", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.14", "stddev": "1.29", "min": "-0.165872422", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Technology"}]}))

MPRe =  ModelPredictionClass(sssalecurrent,7d8c395d8d5a4ff9a32b7597c7013b1e, None, timeseriesforecasting, platformmodelsSarimasstoressarima_20231120-121137timeseriesforecasting, sarima):
responseMetrics = MPRe.getPrediction()
#prediction-pipelinePostExecutionHook(responseMetrics)


except Exception as ex: 
	logging.error(ex)
spark.stop()


***WRITING DATAFRAME***

In [ ]:
#%run superstoressarimapredHooks.ipynb
try:
	#sinkPreExecutionHook()

	superstoressales = NumtraConnector.fetch(spark, "{'samplefile': None, 'samplecount': None, 'originalcount': None, 'orignalKey': None, 'pathOnly': '/Retail And Marketing/SuperStoresSales', 'project_id': '6568d05645a2acea57b5b061', 'parent_id': '6568d05645a2acea57b5b061', 'original_schema': [], 'actual_schema': [], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'SuperStoresSarimaPred.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-810ed0cd-3f4d-4e8d-9952-92285d71fd55-c000.csv', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
